In [17]:
import pandas as pd
from transformers import pipeline

In [18]:
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
labels = ["bussiness","technology","science","sports","media and entertainment","politics","health","crime","accident","environment","art","literature","tragedy","education","fashion","food","travel","military","real estates","history","religion","celebrity"]
hypothesis_template = "this text is about {}"


In [19]:
# data = pd.read_csv('../preprocessed_data/cleaned_data_USA.csv')
# data = pd.read_csv('../preprocessed_data/cleaned_data_Nigeria.csv')
# data = pd.read_csv('../preprocessed_data/cleaned_data_Canada.csv')
# data = pd.read_csv('../preprocessed_data/cleaned_data_Kenya.csv')
# data = pd.read_csv('../preprocessed_data/cleaned_data_Malaysia.csv')
# data = pd.read_csv('../preprocessed_data/cleaned_data_Singapore.csv')
# data = pd.read_csv('../preprocessed_data/cleaned_data_Denmark.csv')
# data = pd.read_csv('../preprocessed_data/cleaned_data_UK.csv')
data = pd.read_csv('../preprocessed_data/cleaned_data_Finland.csv')



dataFrame = data[['date','traffic','newsSnippet']]
strings_list = [item[2] for item in dataFrame.values.tolist()]
loop = range(len(strings_list))



In [20]:
results = []
batch_size = 16
for i in range(0, len(strings_list), batch_size):
    batch = strings_list[i:i+batch_size]
    predictions = classifier(batch, labels, hypothesis_template=hypothesis_template, multi_class=True)
    for j, prediction in enumerate(predictions):
        top_label = prediction['labels'][0]
        top_score = prediction['scores'][0]
        row_data = (dataFrame.iloc[i+j]['date'], dataFrame.iloc[i+j]['traffic'], dataFrame.iloc[i+j]['newsSnippet'], top_label, top_score)
        results.append(row_data)

df_results = pd.DataFrame(results, columns=['date', 'traffic', 'newsTitle', 'predicted_label', 'score'])
df_results.to_csv('./classified_data/regions/europe/classified_data/Finland_classification_output.csv', index=False)

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transf